Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain
gr(size=(500,500));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

### snippet 4.7

In [3]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [4]:
df2 = filter(row -> row[:age] >= 18, df)
mean_weight = mean(df2[:weight])
df2[:weight_c] = convert(Vector{Float64}, df2[:weight]) .- mean_weight ;

Define the Stan language model

In [5]:
weightsmodel = "
data {
 int < lower = 1 > N; // Sample size
 vector[N] height; // Predictor
 vector[N] weight; // Outcome
}

parameters {
 real alpha; // Intercept
 real beta; // Slope (regression coefficients)
 real < lower = 0 > sigma; // Error SD
}

model {
 height ~ normal(alpha + weight * beta , sigma);
}

generated quantities {
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [6]:
stanmodel = Stanmodel(name="weights", monitors = ["alpha", "beta", "sigma"],model=weightsmodel,
  output_format=:mcmcchain);
# Input data for cmdstan
heightsdata = Dict("N" => length(df2[:height]), "height" => df2[:height], "weight" => df2[:weight_c]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME);
# Describe the draws
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean          SD        Naive SE       MCSE      ESS
alpha 154.59975550 0.274401897 0.00433867494 0.0040495051 1000
 beta   0.90357784 0.042493464 0.00067188065 0.0007074509 1000
sigma   5.10557707 0.196570953 0.00310805967 0.0031481204 1000

Quantiles:
          2.5%        25.0%      50.0%      75.0%        97.5%   
alpha 154.07197500 154.419000 154.601500 154.7832500 155.14705000
 beta   0.81823575   0.875814   0.903342   0.9321475   0.98672012
sigma   4.73212350   4.968360   5.095970   5.2334550   5.50913625



Save the chains in a JLS file

In [8]:
serialize("m4.4s.jls", chn)

chn2 = deserialize("m4.4s.jls")

Object of type "Chains{Float64}"

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Union{Missing, Float64}[154.951 0.900715 5.00766; 154.109 0.897911 5.15692; … ; 154.629 0.900685 4.99885; 155.098 0.847303 5.07785]

Union{Missing, Float64}[154.705 0.892157 4.91757; 154.702 0.916396 4.64239; … ; 154.398 0.884192 4.88623; 154.03 0.965689 5.20291]

Union{Missing, Float64}[154.608 0.955013 4.94; 154.466 0.993816 4.94019; … ; 154.402 0.907274 5.09166; 154.91 0.904956 5.11177]

Union{Missing, Float64}[154.758 0.916571 5.20951; 154.545 0.88783 5.5492; … ; 154.617 0.914972 5.08664; 155.125 0.838313 4.80429]

Should be identical to earlier result

In [9]:
describe(chn2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean          SD        Naive SE       MCSE      ESS
alpha 154.59975550 0.274401897 0.00433867494 0.0040495051 1000
 beta   0.90357784 0.042493464 0.00067188065 0.0007074509 1000
sigma   5.10557707 0.196570953 0.00310805967 0.0031481204 1000

Quantiles:
          2.5%        25.0%      50.0%      75.0%        97.5%   
alpha 154.07197500 154.419000 154.601500 154.7832500 155.14705000
 beta   0.81823575   0.875814   0.903342   0.9321475   0.98672012
sigma   4.73212350   4.968360   5.095970   5.2334550   5.50913625



End of `m4.4s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*